---
# this is YAML front matter
toc: true
comments: true
layout: post
title: Linux/Bash Security
description: For our Linux interaction, we decided to get hands-on with some system auditing.
courses: { compsci: { week: 3 }}
categories: [C4.0]
type: hacks
---

# Banner
I made a cool banner using ASCII art and terminal coloring -- this doesn't show up that well in this blog post, so maybe run this script on your own Linux environment. This script also gets some important information about the system, like the OS version, main user, and kernel version.

In [1]:
%%bash


{
version=$(lsb_release -a | grep "Description" | cut -d ":" -f2 | tr -d "	")
} &>/dev/null
user=$(whoami)
kernel=$(uname -r)


echo "Output:"
echo -e "\e[40m
\e[93m                            :             .:                                                                                                          
                           .!           .^5G^.                                                                                                        
                           :Y         ..^?&@Y^..                                                                                                      
                           ~#.            ^!                                                                                                          
                           P@J                                                                                                                        
                         ^G@@@Y.                                                                                                                      
                      :!P@@@@@@#Y~.                                                                                                                   
            .::^~!7?YB&@@@@@@@@@@@&GY?7!~^::..                                                                                                        
                 ...:^!Y#@@@@@@@BJ~:....          .                                                                                                   
                        .?&@@@B~                  ~                                                  .:                                               
                          ^#@P                   :P.                                                 ~?                                               
                           7&.                .:!B@G!:.                                             ^B&!                                              
                           :5             ..:^!Y&@@@&Y!^:..                                   ..:^7G@@@@BJ~::.                                        
                           .7                   .Y@J.                                             .^J&@5~..                                           
                            ^                     Y                                                  7Y                                               
                            .                     ^                                                  .^                                               
                                                  .                                                                                                   
                                                                                                                                                      
                                                                                                                                                      
\e[35m                                       ▄███▄     ▄▄▄▄▀ ▄  █ ▄███▄   █▄▄▄▄ ▄███▄   ██   █                                                              
                                       █▀   ▀ ▀▀▀ █   █   █ █▀   ▀  █  ▄▀ █▀   ▀  █ █  █                                                              
                                       ██▄▄       █   ██▀▀█ ██▄▄    █▀▀▌  ██▄▄    █▄▄█ █                                                              
                                       █▄   ▄▀   █    █   █ █▄   ▄▀ █  █  █▄   ▄▀ █  █ ███▄                                                           
                                       ▀███▀    ▀        █  ▀███▀     █   ▀███▀      █     ▀                                                          
                                                        ▀            ▀              █                                                                 
                                                                                   ▀                                                                  
\e[93m                                                                                                                                                      
                                                                                                                                                      
                                                                                                      ..             .                                
               .:                                                                                     :^             Y~                               
               ~?                                                                                     !!         .:~P@&?:.                            
              ^B&!                                                                                    ?Y            .Y!                               
        ..:^7G@@@@BJ~::.                                                                              B#.            .                                
            .^J&@5~..                                                                               :P@@G:                                            
               7Y                                                                                 :J#@@@@&J:                                          
               .^                                                                         ..::^~!?5B@@@@@@@@@@B5?!~^^:...                             
                            .                                                            ..::^^!7YG&@@@@@@@@&GY7!~^::..                               
                            Y~                                                                    :?#@@@@#?:                .:                        
                        .:~P@&?:.                                                                   .P@@P.                  ~?                        
                           .Y!                                                                        BB                   ^B&!                       
                            .                                                                         ?J             ..:^7G@@@@BJ~::.                 
                                                                                                      ~!                 .^J&@5~..                    
                                                                                                      ::                    7Y                        
                                                                                                      ..                    .^                        
\e[35m                                                 OS: ${version}                                                                                     
                                                 Kernel: ${kernel}                                                                                  
                                                 User: ${user}                                                                             
                                                                                                                                                      
\e[97m                                                                                                                                                      "

Output:

                            :             .:                                                                                                          
                           .!           .^5G^.                                                                                                        
                           :Y         ..^?&@Y^..                                                                                                      
                           ~#.            ^!                                                                                                          
                           P@J                                                                                                                        
                         ^G@@@Y.                                                                                                                      
                      :!P@@@@@@#Y~.                                                  

# Users
This user auditing script does quite a few things to secure every human user on the system. The reason why I specify "human" is that within Linux there are many "system" users that are used for services and system processes. The only users that are required to be "secured" in this manner are human users, or users that were made to log on and function normally.
In the following script I do the following to secure all the human users on the system:
- Changing the password to something secure
- Using chage to set the minimum/maximum days for password expiration
- Changing the login shell to /bin/bash
- Setting correct ownership on the users' home directories

In [ ]:
%%bash


function successOrFail () {
    if [ $? -eq 0 ]
    then
        # Print the success message if it did work
        echo -e $1
        return 0
    else
        # Print the fail message if it didn't work
        echo -e $2
        return 1
    fi
}

password="!_|0v3_@P_c0mput3r_5c13nc3"
# Get a list of all human users (UID >= 1000)
human_users=$(awk -F: '$3 >= 1000 && $1 != "nobody" {print $1}' /etc/passwd)

# Change passwords for each user
for user in $human_users; do
  echo "Changing password for user: $user"
  echo "$user:$password" | chpasswd
  successOrFail "\033[31;32;1m[SUCCESS]\033[0m Successfully changed password of $user" "\033[31;40;1m[FAIL]\033[0m Unable to change password for $user"
  chage -m 7 -M 90 -W 14 $user
  successOrFail "\033[31;32;1m[SUCCESS]\033[0m Chage settings successfully configured for $user" "\033[31;40;1m[FAIL]\033[0m Unable to configure chage for $user"
  chsh -s /bin/bash $user
  successOrFail "\033[31;32;1m[SUCCESS]\033[0m Successfully configured bash login shell for $user" "\033[31;40;1m[FAIL]\033[0m Unable to configure bash login shell for $user"
  chown $user:$user /home/$user/
  successOrFail "\033[31;32;1m[SUCCESS]\033[0m Successfully configured home dir permissions for $user" "\033[31;40;1m[FAIL]\033[0m Unable to configure home dir permissions for $user"
  fi
done

# Exit with success
exit 0